# Notebook to Perform Data Conversion Calculations and Feasibility Adjustments

In [ ]:
import pandas as pd

# Read Files
demanders = pd.read_excel('Busses_Timeseries_Table.xlsx')
generators = pd.read_excel('Generators_Timeseries_Table_.xlsx')
generatorsGrid = pd.read_excel('Generators_Table.xlsx')
branches = pd.read_excel('Branches_Input_Table.xlsx')

### Factorize Consumer/Bus Index

In [ ]:
# Adjust bus id and Reduce bus number by 1 so in {0.. n_bus-1}
demanders['bus_id'] = pd.factorize(demanders['load_area'])[0]
branches['To Number'] =  branches['To Number'] - 1
branches['From Number'] =  branches['From Number'] - 1
generatorsGrid['Number of Bus'] =  generatorsGrid['Number of Bus'] - 1

### Scale Edge Capacities

In [ ]:
scalingFactor = 10
branches['Lim MVA A'] = branches['Lim MVA A']*scalingFactor

### Scale Generator Capacity

Adapt to not exceed edge limits - see cleaning

### Normalize Demand

In [ ]:
import random

def capacitySum (timeInterval):
    index = 0
    sum = 0
    while index < 6:
        generator = generators.iloc[[timeInterval * 6 + index]]
        sum += generator['max_ecomax'].item()
        index += 1
    #Add randomness to not fill capacity every time
    gap = random.uniform(0, 15) / 100 # small to not make infeasible with round up
    sum = sum* (1-gap)
    return sum

In [ ]:
# Formula demandi/SumDemand * CapacitySum = zi
# CapacitySum * (1- gap) to not fill capacity completely

import numpy as np
import math

#30 demanders in time interval
demandersSize = 30
timeInterval = 0

while timeInterval < len(demanders.index) / demandersSize:
    index = 0
    demandSum = 0

    while index < demandersSize:
        row = demanders.iloc[[timeInterval * demandersSize + index]]
        demandSum += row['mw'].item()
        index += 1
        
    capacitySum = capacitySum(timeInterval)
    #if timeInterval == 0:
        #print("Capacity: " + str(capacitySum) + " Sum: " + str(demandSum))
    index = 0
    if sum > capacitySum: 
        while index < demandersSize:
            row = demanders.iloc[[timeInterval * 30 + index]]
            xi = row['mw'].item()

            zi = xi/ demandSum * capacitySum

            #round to two digits after comma
            demanders.at[timeInterval * demandersSize + index, 'mw_norm'] = round(zi, 2)
            #if timeInterval == 0:
                #print(str(timeInterval) + ': zi ' + str(demanders.at[timeInterval * 30 + index, 'mw_norm']))
            index += 1
    timeInterval += 1

### Calculate Marginal Prices

In [ ]:
# Find the bmin-index and bmax-index of a generator bid from the given values
def find_b (row):
    bmin = -1
    bmax = -1
    for columnIndex, value in row.items():
        if bmin == -1 and columnIndex.startswith('mw') and pd.notna(value): # First given value
            bmin = columnIndex[2:len(columnIndex)]
        if bmin != -1 and columnIndex.startswith('mw') and pd.isna(value): # Last given value
            bmax = int(columnIndex[2:len(columnIndex)]) -1
            break
    bmin = int(bmin)
    return bmin, bmax

In [ ]:
# C_pw(bmax) of each generator bid

def price_calc (row):
    bmin, bmax = find_b(row) 
    #print("Bmax: " + str(bmax) + " bmin: " + str(bmin))
    if bmax == -1 or bmin == -1:
        return 0 # Return 0, translates to x = 0
    
    bidname = "bid" + str(bmin)
    mwname = "mw" + str(bmin) 
    mwmax = row[mwname]
    if mwmax == 0:
        mwmax = 1
        
    cost = row[bidname] * row[mwname] # First cost without factorization
    #print("First cost: " + str(cost) + "from mw:" + str(row[mwname]) + "and bid:" + str(row[bidname]))

    bindex = bmin + 1
    while bindex <= bmax:
        bidname = "bid" + str(bindex)
        mwname = "mw" + str(bindex)
        mwnameMinus1 = "mw" + str(bindex - 1)
        
        cost += row[bidname] * (row[mwname] - row[mwnameMinus1])

        if row['bid_slope_flag'] == 'True': # Factorization if flag= True
            bidnameMinus1 = "bid" + str(bindex -1)
            cost += ((row[bidname] - row[bidnameMinus1])*(row[mwname] - row[mwnameMinus1]))/2
                 
        mwmax = row[mwname]
        #print("My cost" + str(cost) + "mwmax:" + str(mwmax)) 
        bindex += 1
    
    print("Final: " + str(cost/mwmax))
    return cost/mwmax  # To get price not cost   

In [ ]:
generators['incremental_price'] = generators.apply (lambda row: price_calc(row), axis = 1)